# Importing the required packages


In [1]:
import requests
from selenium import webdriver
import bs4
import re
from tqdm import tqdm
import gc
import pandas as pd

In [2]:
# web driver for selenium
driver = webdriver.Chrome(executable_path=r'C:\Webdriver\chromedriver.exe')

# get the web page and store it as a soup element
driver.get('https://www.hdfcbank.com/personal/pay/cards/credit-cards')
soup = bs4.BeautifulSoup(driver.page_source, 'lxml')
driver.close()

Required info:

[Card Name, Card fee, Reward points/percentage per 100 spent, Lounge access, Milestone benefit, Card fee reversal condition if any]

In [3]:
# Here we are extracting the card names and the links to the feature pages
card_names = []
links = []
for i in soup.findAll("div",{"class":"cardparent"}):
    
    # From the parent class we need to extract the title/card name 
    s = i.findAll('h2',{'class':'cardTitle'})[0]
    card_names.append(s.get_text('^^',strip=True).split('^^')[0])
    
    # Now we will find the links to the know more section and store them
    for link in i.findAll("a",{'href': re.compile(r'/personal'),'target':'_self'}):
        a = link.get('href')
        if a!=None:
            links.append(a)
            break

# Printing out some of the card names and their linnk
for name, link in zip(card_names[:5], links[:5]):
    print(name,":",link)

MoneyBack+ Credit Card : /personal/pay/cards/credit-cards/moneyback-plus
Millennia Credit Card : /personal/pay/cards/millennia-cards/millennia-cc-new
IndianOil HDFC Bank Credit Card : /personal/pay/cards/credit-cards/indianoil-hdfc-bank-credit-card
Regalia Gold Credit Card : /personal/pay/cards/credit-cards/regalia-gold-credit-card
IRCTC HDFC Bank Credit Card : /personal/pay/cards/credit-cards/irctc-credit-card


In [4]:
# From the above obtained links, creating and storing the complete links to the web page
c_links = [f"https://www.hdfcbank.com{link}" for link in links]
print(*c_links[0:5],sep='\n')

https://www.hdfcbank.com/personal/pay/cards/credit-cards/moneyback-plus
https://www.hdfcbank.com/personal/pay/cards/millennia-cards/millennia-cc-new
https://www.hdfcbank.com/personal/pay/cards/credit-cards/indianoil-hdfc-bank-credit-card
https://www.hdfcbank.com/personal/pay/cards/credit-cards/regalia-gold-credit-card
https://www.hdfcbank.com/personal/pay/cards/credit-cards/irctc-credit-card


In [5]:
# Creating a function that retrieves the content of a webpage given the link
def get_page(link):
    """
    This function return the bs4 format for a given link.
    """
    driver = webdriver.Chrome(executable_path=r'C:\Webdriver\chromedriver.exe')
    driver.get(link)
    soup_res = bs4.BeautifulSoup(driver.page_source, 'lxml')
    driver.close()
    
    return soup_res

## Extracting all the other information for each of the credit cards 

In [6]:
# empty lists to store the extracted information
all_features = []
all_points = []
lounge_access = []
milestone_benefit = []
rewards = []
opt_s = []

# Iterating over all the links
for link in c_links:
    
    # get the webpage for the key-features
    key_features = get_page(link)
    
    features = []      # to store the points available in the know more page of each credit card
    fpoints = []       # To store the sub-points corresponding to each point

    # iterate throung each point which are present in the content-body
    for i in key_features.findAll("div",{"class":"content-body"}):
        
        # get the names for the points/features
        s = i.findAll('h4',{'class':'row-name'})  # This return a list of elements
        
        # get the corresponding list of sub-points or paragraph
        points = i.findAll(['li','p'])
        
        tmp_points = []     # This is to store all the subpoints
        for point in points:
            tmp_points.append(point.get_text(strip=True))
        fpoints.append(tmp_points)
        del tmp_points; gc.collect()

        try:
            # extract the features and store it
            features.append(s[0].get_text('^^',strip=True))

        except Exception as e:
            pass
    
    
    # Remove special characters from the feature names
    features = [re.sub(r'[^a-zA-Z0-9 ]+',"",f).lower() for f in features]
    
    # create a dictionary for the different features and the feature points
    data = {f:x for f,x in zip(features, fpoints)}
    
    # Filter the features required to us
    filtered_featues = list(filter(lambda x: re.search('feature\w*|mile\w*|reward\w*|lounge\w*|benefit\w+',x), features))
    
    # Lounge access:
    lounge_key = list(filter(lambda x:True if re.search(r'lounge',x) else False,filtered_featues))
    if len(lounge_key)!=0:
        lounge_access.append(data[lounge_key[0]])
    else:
        lounge_access.append('None')
    
    # Milestone benefit:
    mile_key = list(filter(lambda x:True if re.search(r'mile\w*',x) else False, filtered_featues))
    if len(mile_key)!=0:
        milestone_benefit.append(data[mile_key[0]])
    else:
        milestone_benefit.append('None')
    
    # rewards
    rew_key = list(filter(lambda x:True if re.search(r'reward\w*|key\w*|feature\w*',x) else False,filtered_featues))
    if len(rew_key)==1:
        rewards.append(data[rew_key[0]])
    elif len(rew_key)>1:
        tmp = []
        for rk in rew_key:
            tmp.extend(data[rk])
        rewards.append(tmp)
    else:
        rewards.append('None')
    
    # get the webpage for fees and charges
    fee_ = get_page(link+'/fees-and-charges')
    s = []
    for i in fee_.findAll('div',{'class':'inner-content'}):
        s_ = i.findAll(['li','p'])
        s.extend([y.get_text('^^',strip=True) for y in s_])
    
    # renewal offers also contain details of card fee reversal, so we are going to append that information to fee
    ren_off = list(filter(lambda x:True if re.search(r'renew\w*',x) else False,filtered_featues))
    if len(ren_off)!=0:
        s.extend(ren_off[0])
    opt_s.append(s)

In [63]:
# converting the obtained results to a dictionary and then storing it in a pandas dataframe
xyz = {"card_name":card_names,
    "lounge_access":lounge_access,
        "milestone_benefit":milestone_benefit,
        "rewards":rewards,
       "fees":opt_s}
df = pd.DataFrame.from_dict(xyz)
df = df.drop_duplicates(subset = ['card_name'])
# df.to_csv('all_card_details.csv',index=False)
df.head()

,card_name,lounge_access,milestone_benefit,rewards,fees
0,MoneyBack+ Credit Card,None,None,"[10X CashPoints on Amazon, BigBasket , Flipkar...",[Joining/Renewal Membership Fee – ₹500/- + App...
1,Millennia Credit Card,[8 Complimentary Domestic Lounge Access per ca...,None,"[5% Cashback on Amazon, BookMyShow, Cult.fit, ...","[Joining/Renewal Membership Fee – ₹1,000/- + A..."
2,IndianOil HDFC Bank Credit Card,None,None,"[Fuel Points, , , Earn up to 50 Litres of Free...",[Joining/Renewal Membership Fee – Rs. 500/- pl...
3,Regalia Gold Credit Card,[A. Complimentary Airport Lounge Access within...,"[Get ₹ 1,500 worth vouchers from Marriott, Myn...",[Complimentary Club Vistara Silver Tier and MM...,[Joining/ Renewal Membership Fee of Rs. 2500 +...
4,IRCTC HDFC Bank Credit Card,[8 complimentary access to select IRCTC Execut...,[Gift voucher worth INR 500 on quarterly spend...,"[HDFC Bank Reward Points, 5 Reward Point for e...",[Joining/Renewal Membership Fee – Rs. 500/- + ...


## Extracting information on Card Fees and Card Fee reversal condition

In [64]:
card_fees = []
card_rev_cond = []
for s in df.fees.values:
    s = set(s)
    l = []
    for i in s:
        l.extend(i.split('^^'))
    
    # filter statements based on certain keywords
    reversal_cond = list(filter(lambda x:True if re.search(r'waiv\w*|membership\s*\w*\s*free|spend\w*',x.lower()) else False, l))
    reversal_cond = list(map(lambda x:re.sub(r'(\xa0)*|(\u200b)*',"",x).strip(), reversal_cond))
    
    if len(reversal_cond)!=0:
        card_rev_cond.append(';'.join(reversal_cond))
    else:
        card_rev_cond.append('None')
    
    # Cascade conditions to filter out what we need exactly
    
    # 1. Find statements having digits
    l = list(filter(lambda x:True if re.search(r'\d+',x.lower()) else False, l))
    
    # Find statements that do not contain the keyword waiv\w+ meaning waiver/waived etc
    l = list(filter(lambda x:False if re.search(r'waiv\w+',x.lower()) else True, l))
    
    # Finally find the staments containing certain patterns
    pattern = r'nominal|join\w*|minimum|renewal membership|per annum|monthly membership fee|annual membership fee'
    fees = list(filter(lambda x:True if re.search(pattern,x.lower()) else False, l))
    
    # If we have multiple statements we will join them together
    if len(fees)!=0:
        card_fees.append(';'.join(fees))
    else:
        card_fees.append('None')
    

In [65]:
# # Add the extracted details to the dataframe
df['card_fees'] = card_fees
df['card_rev_cond'] = card_rev_cond
df.head()

,card_name,lounge_access,milestone_benefit,rewards,fees,card_fees,card_rev_cond
0,MoneyBack+ Credit Card,None,None,"[10X CashPoints on Amazon, BigBasket , Flipkar...",[Joining/Renewal Membership Fee – ₹500/- + App...,Joining/Renewal Membership Fee – ₹500/- + Appl...,"Spend ₹50,000 or more in a year, before your C..."
1,Millennia Credit Card,[8 Complimentary Domestic Lounge Access per ca...,None,"[5% Cashback on Amazon, BookMyShow, Cult.fit, ...","[Joining/Renewal Membership Fee – ₹1,000/- + A...","Joining/Renewal Membership Fee – ₹1,000/- + Ap...","Spend ₹1,00,000 or more in a year, before your..."
2,IndianOil HDFC Bank Credit Card,None,None,"[Fuel Points, , , Earn up to 50 Litres of Free...",[Joining/Renewal Membership Fee – Rs. 500/- pl...,Joining/Renewal Membership Fee – Rs. 500/- plu...,None
3,Regalia Gold Credit Card,[A. Complimentary Airport Lounge Access within...,"[Get ₹ 1,500 worth vouchers from Marriott, Myn...",[Complimentary Club Vistara Silver Tier and MM...,[Joining/ Renewal Membership Fee of Rs. 2500 +...,Joining/ Renewal Membership Fee of Rs. 2500 + ...,None
4,IRCTC HDFC Bank Credit Card,[8 complimentary access to select IRCTC Execut...,[Gift voucher worth INR 500 on quarterly spend...,"[HDFC Bank Reward Points, 5 Reward Point for e...",[Joining/Renewal Membership Fee – Rs. 500/- + ...,Joining/Renewal Membership Fee – Rs. 500/- + A...,"Spend Rs. 1,50,000 or more in a year, before y..."


Required info:
[Card Name, Card fee, Reward points/percentage per 100 spent, Lounge access, Milestone benefit, Card fee reversal condition if any]

## Further extraction/filtering of information on rewards 

In [66]:
fil_rew_val = []
for rew in  df.rewards.values:
#     print('raw : ',rew)
    if rew != None:
        
        # Filter out the statements having digits in them
        l = list(filter(lambda x:True if re.search(r'\d+',x.lower()) else False, rew))
        l = set(l)
        
        # Filter our statements having specific keywords
        l = list(filter(lambda x:False if re.search(r'(rewardpoint|rp) =',x.lower()) else True, l))
        l = list(filter(lambda x:True if re.search(r'rewards|point|reward point\w*|cashpoints',x.lower()) else False, l))
        
        # removing extra unnecessary characters
        l = set(map(lambda x:re.sub(r'(\xa0)*|(\u200b)*',"",x).strip(), l))
        l = list(l)
        
        # Filter out statements where there are conditions given
        l = list(filter(lambda x:False if re.search(r'\w*with effect\w*',x.lower()) else True, l))
        
        # Store the filtered list
        fil_rew_val.append(';'.join(l))
    else:
        fil_rew_val.append('None')
    

df.rewards = fil_rew_val
df.head()

,card_name,lounge_access,milestone_benefit,rewards,fees,card_fees,card_rev_cond
0,MoneyBack+ Credit Card,None,None,CashPoints earned on your HDFC Bank MoneyBack+...,[Joining/Renewal Membership Fee – ₹500/- + App...,Joining/Renewal Membership Fee – ₹500/- + Appl...,"Spend ₹50,000 or more in a year, before your C..."
1,Millennia Credit Card,[8 Complimentary Domestic Lounge Access per ca...,None,"Flights and Hotels redemption, Credit Card mem...","[Joining/Renewal Membership Fee – ₹1,000/- + A...","Joining/Renewal Membership Fee – ₹1,000/- + Ap...","Spend ₹1,00,000 or more in a year, before your..."
2,IndianOil HDFC Bank Credit Card,None,None,Redeem Fuel Points for Free Fuel using the com...,[Joining/Renewal Membership Fee – Rs. 500/- pl...,Joining/Renewal Membership Fee – Rs. 500/- plu...,None
3,Regalia Gold Credit Card,[A. Complimentary Airport Lounge Access within...,"[Get ₹ 1,500 worth vouchers from Marriott, Myn...",Earn 20 Reward Points on every Rs.150 retail* ...,[Joining/ Renewal Membership Fee of Rs. 2500 +...,Joining/ Renewal Membership Fee of Rs. 2500 + ...,None
4,IRCTC HDFC Bank Credit Card,[8 complimentary access to select IRCTC Execut...,[Gift voucher worth INR 500 on quarterly spend...,The redemption against the train ticket bookin...,[Joining/Renewal Membership Fee – Rs. 500/- + ...,Joining/Renewal Membership Fee – Rs. 500/- + A...,"Spend Rs. 1,50,000 or more in a year, before y..."


## Extract/Filter Info from lounge access column

In [67]:
df.lounge_access.values

array(['None',
       list(['8 Complimentary Domestic Lounge Access per calendar year (2 in a quarter)Diners Club International Lounge ProgramClick hereClick Hereto check the list of lounges for other variants', 'For Mastercard Cards, Rs.25 will be charged and immediately made void at the terminal for verification.', 'For Visa/Diners Cards, Rs.2 will be charged to the card account as a transaction for verification']),
       'None',
       list(['A. Complimentary Airport Lounge Access within India', '12 complimentary access within India (both at domestic and international terminals) using the HDFC Bank Regalia Gold Credit Card.', 'Swipe your HDFC Bank Regalia Gold Credit Card at the lounge to avail the complimentary lounge access', 'Click hereto check the list of lounges', 'Visits exceeding the complimentary quota & Lounges not covered under the programme will be allowed at the discretion of lounge and will also be chargeable by the Lounge.', 'B. Priority Pass', 'Using Priority Pass, y

In [68]:
la = []
for i in df.lounge_access.values:
    
    # Filter out statements having digits or the word unlimited
    l = list(filter(lambda x:True if re.search(r'\d+|unlimited',x.lower()) else False, i))
    
    #
    l = list(filter(lambda x:False if re.search(r'\s*\w*click\w*\s*',x.lower()) else True, l))
    
    # Filter out statements having lounge access or complimentary
    l = list(filter(lambda x:True if re.search(r'lounge access|complimentary',x.lower()) else False, l))
    
    # Remove special characters
    l = set(map(lambda x:x.replace('\xa0','').replace('\u200b',' ').strip(), l))
    l = list(l)
    if len(l)!=0:
        la.append(';'.join(l))
    else:
        la.append('None')

# append the results to lounge access
df.lounge_access = la

## Extract/filter info from milestone benefits column

In [69]:
mb_vals = []
for mb in df.milestone_benefit.values:
    if mb!='None':
        l = set(map(lambda x:x.replace('\xa0','').replace('\u200b',' ').strip(), mb))
        l = list(filter(lambda x:False if re.search(r'\s*\w*click\w*\s*',x.lower()) else True, l))
        mb_vals.append(';'.join(l))
    else:
        mb_vals.append('None')
df.milestone_benefit = mb_vals

In [70]:
df.head()

,card_name,lounge_access,milestone_benefit,rewards,fees,card_fees,card_rev_cond
0,MoneyBack+ Credit Card,None,None,CashPoints earned on your HDFC Bank MoneyBack+...,[Joining/Renewal Membership Fee – ₹500/- + App...,Joining/Renewal Membership Fee – ₹500/- + Appl...,"Spend ₹50,000 or more in a year, before your C..."
1,Millennia Credit Card,None,None,"Flights and Hotels redemption, Credit Card mem...","[Joining/Renewal Membership Fee – ₹1,000/- + A...","Joining/Renewal Membership Fee – ₹1,000/- + Ap...","Spend ₹1,00,000 or more in a year, before your..."
2,IndianOil HDFC Bank Credit Card,None,None,Redeem Fuel Points for Free Fuel using the com...,[Joining/Renewal Membership Fee – Rs. 500/- pl...,Joining/Renewal Membership Fee – Rs. 500/- plu...,None
3,Regalia Gold Credit Card,"Using Priority Pass, you and your add-on membe...",";Get ₹ 1,500 worth vouchers from Marriott, Myn...",Earn 20 Reward Points on every Rs.150 retail* ...,[Joining/ Renewal Membership Fee of Rs. 2500 +...,Joining/ Renewal Membership Fee of Rs. 2500 + ...,None
4,IRCTC HDFC Bank Credit Card,None,,The redemption against the train ticket bookin...,[Joining/Renewal Membership Fee – Rs. 500/- + ...,Joining/Renewal Membership Fee – Rs. 500/- + A...,"Spend Rs. 1,50,000 or more in a year, before y..."


In [71]:
# Drop any duplicate rows if any
df = df.drop_duplicates(subset=['card_name'])
df = df[['card_name','card_fees','rewards','lounge_access', 'milestone_benefit', 'card_rev_cond']]
df.head()

,card_name,card_fees,rewards,lounge_access,milestone_benefit,card_rev_cond
0,MoneyBack+ Credit Card,Joining/Renewal Membership Fee – ₹500/- + Appl...,CashPoints earned on your HDFC Bank MoneyBack+...,None,None,"Spend ₹50,000 or more in a year, before your C..."
1,Millennia Credit Card,"Joining/Renewal Membership Fee – ₹1,000/- + Ap...","Flights and Hotels redemption, Credit Card mem...",None,None,"Spend ₹1,00,000 or more in a year, before your..."
2,IndianOil HDFC Bank Credit Card,Joining/Renewal Membership Fee – Rs. 500/- plu...,Redeem Fuel Points for Free Fuel using the com...,None,None,None
3,Regalia Gold Credit Card,Joining/ Renewal Membership Fee of Rs. 2500 + ...,Earn 20 Reward Points on every Rs.150 retail* ...,"Using Priority Pass, you and your add-on membe...",";Get ₹ 1,500 worth vouchers from Marriott, Myn...",None
4,IRCTC HDFC Bank Credit Card,Joining/Renewal Membership Fee – Rs. 500/- + A...,The redemption against the train ticket bookin...,None,,"Spend Rs. 1,50,000 or more in a year, before y..."


In [72]:
df.to_csv('all_card_details.csv',index=False)